In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from gptchem.data import get_moosavi_mof_data
from sklearn.model_selection import train_test_split
from gptchem.baselines.henry import FEATURES
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from tabpfn.scripts.transformer_prediction_interface import TabPFNClassifier

import pandas as pd

In [2]:
data = get_moosavi_mof_data()

In [9]:
data['logKH_CH4_bins'] = pd.qcut(data['logKH_CH4'], 5, labels=[0, 1, 2, 3, 4])

In [14]:
train_data, test_data = train_test_split(data, train_size=100)

In [15]:
len(train_data)

100

In [16]:
X_train, y_train = train_data[FEATURES], train_data["logKH_CH4_bins"]
X_test, y_test = test_data[FEATURES], test_data["logKH_CH4_bins"]

In [17]:
selector = SelectFromModel(estimator=RandomForestClassifier(), max_features=100)
X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)
tabpfn = TabPFNClassifier(device="cpu", N_ensemble_configurations=32)

tabpfn.fit(X_train, y_train)

tabpfn_predictions, _ = tabpfn.predict(X_test, return_winning_probability=True)

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
